In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from gammapy.data import DataStore
from gammapy.image import SkyImage, SkyImageList, SkyMask
from gammapy.scripts import StackedObsImageMaker
from gammapy.utils.energy import Energy

In [ ]:
#data_store containing the observations you will stack in the cube
dataset_directory="$GAMMAPY_EXTRA/test_datasets/cube/data"
#cwd= cwd=Path.cwd()
#dataset_directory=str(cwd)+"/data"
data_store = DataStore.from_dir(dataset_directory)

center = SkyCoord(83.63, 22.01, unit='deg').galactic
energy_band = Energy([1, 10], 'TeV')
offset_band = Angle([0, 2.49], 'deg')
    

ref_image = SkyImage.empty(nxpix=50, nypix=50, binsz=0.02, xref=center.l.deg,
                               yref=center.b.deg, proj='TAN', coordsys='GAL')

exclusion_mask = SkyMask.read('$GAMMAPY_EXTRA/datasets/exclusion_masks/tevcat_exclusion.fits')
exclusion_mask = exclusion_mask.reproject(reference=ref_image)

# Pb with the load psftable for one of the run that is not implemented yet...
data_store.hdu_table.remove_row(14)

image_maker = StackedObsImageMaker(
        empty_image=ref_image, energy_band=energy_band, offset_band=offset_band, data_store=data_store,
        obs_table=data_store.obs_table, exclusion_mask=exclusion_mask,
    )
image_maker.make_images(make_background_image=True, for_integral_flux=True, radius=10.)
images = image_maker.images


In [ ]:
#plt.imshow(images["counts"].data,interpolation='None')
images["counts"].plot()
